In [1]:
from datetime import datetime
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
import csv

img_size = 128
img_nr_to_train = 3000
batch_size = 100
epochs_to_fit = 10

In [2]:
# Define the model

model = models.Sequential()
model.add(layers.Conv2D(32, 3, padding="same", activation="relu", input_shape=(img_size, img_size, 1)))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(32, 3, padding="same", activation="relu"))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(64, 3, padding="same", activation="relu"))
model.add(layers.MaxPool2D())
#model.add(layers.Dropout(0.4))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(4, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0

In [3]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint_filepath = 'D:\Repo_Licenta\Models'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)


In [4]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]

(None, 128, 128, 1) <dtype: 'float32'>
(None, 4) <dtype: 'float32'>


[None]

In [5]:
# labels = ['Cover', 'JMiPOD', 'JUNIWARD', 'UERD']
# data_dir = r'D:\Repo_Licenta\resized_grayscale_alaska2'
labels = ['dct', 'lsb_grayscale', 'ssb4', 'ssbn']
data_dir = r'D:\Repo_Licenta\resized_grayscale_digital_steganography'

for batch in range(img_nr_to_train // batch_size):
    print("Batch:", batch + 1)
    x_train = []
    y_train = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        dir = os.listdir(path)
        for index in range(batch * batch_size, (batch + 1) * batch_size):
            img = dir[index]
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            x_train.append(img_arr)
            label = [0, 0, 0, 0]
            label[class_num] = 1
            y_train.append(label)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = x_train[..., np.newaxis]
    x_train.reshape(-1, img_size, img_size, 1)
    p = np.random.permutation(len(x_train))
    x_train = x_train[p]
    y_train = y_train[p]
    history = model.fit(x_train, y_train, epochs=epochs_to_fit, verbose=1, callbacks=[model_checkpoint_callback])


Batch: 1
Epoch 1/10
13/13 [==============================] - 4s 91ms/step - loss: 29.6431 - accuracy: 0.2175

In [7]:
model.load_weights(checkpoint_filepath)

In [22]:
test_dir = r'D:\Repo_Licenta\resized_grayscale_alaska2\Test'
x_test = []

for img in os.listdir(test_dir):
    img_arr = cv2.imread(os.path.join(test_dir, img), cv2.IMREAD_GRAYSCALE)
    x_test.append(img_arr)

In [23]:
x_test = np.array(x_test)
x_test = x_test[..., np.newaxis]
x_test.reshape(-1, img_size, img_size, 1)
result = model.predict(x_test, verbose=1)
print(result.shape)
result

157/157 [==============================] - 3s 20ms/step

array([[0.25249544, 0.24960886, 0.24863088, 0.24926487],
       [0.2519604 , 0.24970922, 0.24891306, 0.2494173 ],
       [0.25273922, 0.24962439, 0.24845472, 0.24918169],
       ...,
       [0.25148988, 0.24993838, 0.24905191, 0.24951984],
       [0.25125015, 0.2497087 , 0.24935687, 0.24968433],
       [0.24888317, 0.24818154, 0.25357872, 0.2493566 ]], dtype=float32)

157/157 [==============================] - 3s 19ms/step

array([[0.24985842, 0.25057757, 0.25037214, 0.24919187],
       [0.24985842, 0.25057757, 0.25037214, 0.24919187],
       [0.24985842, 0.25057757, 0.25037214, 0.24919187],
       ...,
       [0.24985842, 0.25057757, 0.25037214, 0.24919187],
       [0.24985842, 0.25057757, 0.25037214, 0.24919187],
       [0.24985842, 0.25057757, 0.25037214, 0.24919187]], dtype=float32)

In [24]:
# now = datetime.now()
# # dd/mm/YY H:M:S
# dt_string = now.strftime("%d/%m/%Y_%H:%M:%S")
# model.save(fr'D:\Repo_Licenta\Models\{dt_string}')

In [25]:
# for batch in range(5000 // batch_size):
#     print("Batch:", batch)
#     x_train = []
#     y_train = []
#     for label in labels:
#         path = os.path.join(data_dir, label)
#         class_num = labels.index(label)
#         for index in range(batch * batch_size, (batch + 1) * batch_size):
#             try:
#                 #img = os.listdir(path)[index]
#                 img_arr = cv2.imread(os.path.join(path, os.listdir(path)[index]), cv2.IMREAD_GRAYSCALE)
#                 x_train.append(img_arr)
#                 y_train.append(class_num)
#             except Exception as e:
#                 print(e)
#     x_train = np.array(x_train) / 255
#     x_train = x_train[..., np.newaxis]
#     x_train.reshape(-1, img_size, img_size, 1)
#     y_train = np.array(y_train)
#     p = np.random.permutation(len(x_train))
#     x_train = x_train[p]
#     y_train = y_train[p]
#     history = model.fit(x_train, y_train, epochs=3, verbose=1)
# acc = history.history['accuracy']
# loss = history.history['loss']
#
# epochs_range = range(3)
#
# plt.figure(figsize=(15, 15))
# plt.subplot(2, 1, 1)
# plt.plot(epochs_range, acc, label='Training Accuracy')
# plt.legend(loc='lower right')
# plt.title('Training Accuracy')
#
# plt.subplot(2, 1, 2)
# plt.plot(epochs_range, loss, label='Training Loss')
# plt.legend(loc='upper right')
# plt.title('Training Loss')
# plt.show()
